In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report


In [2]:
train_data_dir = "./data/train"
test_data_dir = "./data/test"

image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    validation_split=0.2
)

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 303 images belonging to 2 classes.


In [4]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 75 images belonging to 2 classes.


In [5]:
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return precision, recall


model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
class MetricsCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_loss, val_accuracy = self.model.evaluate(validation_generator)
        val_predictions = self.model.predict(validation_generator)
        val_true_labels = validation_generator.classes
        val_predicted_labels = val_predictions.argmax(axis=-1)
        val_precision, val_recall = calculate_metrics(val_true_labels, val_predicted_labels)
        print(
            f'Epoch {epoch + 1} - Validation Loss: {val_loss:.4f} - Validation Accuracy: {val_accuracy:.4f} - Validation Precision: {val_precision:.4f} - Validation Recall: {val_recall:.4f}')


metrics_callback = MetricsCallback()


In [7]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[metrics_callback]
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Epoch 1/10
3/3 [==============================] - 1s 151ms/step
Epoch 1 - Validation Loss: 0.6911 - Validation Accuracy: 0.4000 - Validation Precision: 0.5803 - Validation Recall: 0.5067
9/9 [==============================] - 7s 713ms/step - loss: 3.1756 - accuracy: 0.5018 - val_loss: 0.6806 - val_accuracy: 0.3906
Epoch 2/10
3/3 [==============================] - 1s 142ms/step
Epoch 2 - Validation Loss: 0.5999 - Validation Accuracy: 0.7067 - Validation Precision: 0.4839 - Validation Recall: 0.5200
9/9 [==============================] - 6s 661ms/step - loss: 0.4509 - accuracy: 0.8635 - val_loss: 0.5905 - val_accuracy: 0.7188
Epoch 3/10
3/3 [==============================] - 1s 131ms/step
Epoch 3 - Validation Loss: 1.1127 - Validation Accuracy: 0.5467 - Validation Precision: 0.2988 - Validation Recall: 0.5467
9/9 [==============================] - 6s 689ms/step - loss: 0.2247 - accuracy: 0.9299 - val_loss: 1.2154 - val_accuracy: 0.5156
Epoch 4/10


/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3/3 [==============================] - 1s 138ms/step
Epoch 4 - Validation Loss: 0.2701 - Validation Accuracy: 0.8800 - Validation Precision: 0.4743 - Validation Recall: 0.4933
9/9 [==============================] - 6s 666ms/step - loss: 0.1884 - accuracy: 0.9225 - val_loss: 0.2879 - val_accuracy: 0.8281
Epoch 5/10
3/3 [==============================] - 1s 146ms/step
Epoch 5 - Validation Loss: 0.8252 - Validation Accuracy: 0.6000 - Validation Precision: 0.5269 - Validation Recall: 0.5467
9/9 [==============================] - 6s 675ms/step - loss: 0.1928 - accuracy: 0.9483 - val_loss: 0.8260 - val_accuracy: 0.5625
Epoch 6/10
3/3 [==============================] - 1s 162ms/step
Epoch 6 - Validation Loss: 1.0636 - Validation Accuracy: 0.5467 - Validation Precision: 0.2988 - Validation Recall: 0.5467
9/9 [==============================] - 6s 678ms/step - loss: 0.1301 - accuracy: 0.9631 - val_loss: 1.1701 - val_accuracy: 0.5156
Epoch 7/10


/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3/3 [==============================] - 1s 158ms/step
Epoch 7 - Validation Loss: 0.5747 - Validation Accuracy: 0.6400 - Validation Precision: 0.5527 - Validation Recall: 0.5600
9/9 [==============================] - 7s 754ms/step - loss: 0.1249 - accuracy: 0.9557 - val_loss: 0.7051 - val_accuracy: 0.6562
Epoch 8/10
3/3 [==============================] - 1s 159ms/step
Epoch 8 - Validation Loss: 0.3925 - Validation Accuracy: 0.7733 - Validation Precision: 0.4216 - Validation Recall: 0.4800
9/9 [==============================] - 6s 714ms/step - loss: 0.1469 - accuracy: 0.9483 - val_loss: 0.3965 - val_accuracy: 0.7812
Epoch 9/10
3/3 [==============================] - 1s 160ms/step
Epoch 9 - Validation Loss: 0.3742 - Validation Accuracy: 0.7067 - Validation Precision: 0.3945 - Validation Recall: 0.4400
9/9 [==============================] - 6s 745ms/step - loss: 0.1676 - accuracy: 0.9446 - val_loss: 0.3482 - val_accuracy: 0.7500
Epoch 10/10
3/3 [==============================] - 1s 164ms/ste

/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
test_loss, test_accuracy = model.evaluate(test_generator)

predictions = model.predict(test_generator)
for i, prediction in enumerate(predictions):
    confidence_drowsy = prediction[0]
    confidence_not_drowsy = prediction[1]
    print(
        f"Image {i + 1} - Confidence Drowsy: {confidence_drowsy:.2f}, Confidence Not Drowsy: {confidence_not_drowsy:.2f}")


8/8 [==============================] - 1s 158ms/step
Image 1 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 2 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 3 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 4 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 5 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 6 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 7 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 8 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 9 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 10 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 11 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 12 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 13 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 14 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0.00
Image 15 - Confidence Drowsy: 1.00, Confidence Not Drowsy: 0

In [10]:
test_predictions = model.predict(test_generator)
test_true_labels = test_generator.classes
test_predicted_labels = test_predictions.argmax(axis=-1)
test_precision, test_recall = calculate_metrics(test_true_labels, test_predicted_labels)

print(
    f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.4f} - Test Precision: {test_precision:.4f} - Test Recall: {test_recall:.4f}")


8/8 [==============================] - 3s 352ms/step
Test Loss: 7.3924 - Test Accuracy: 0.3834 - Test Precision: 0.1470 - Test Recall: 0.3834


/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
train_predictions = model.predict(train_generator)
train_true_labels = train_generator.classes
train_predicted_labels = train_predictions.argmax(axis=-1)

validation_predictions = model.predict(validation_generator)
validation_true_labels = validation_generator.classes
validation_predicted_labels = validation_predictions.argmax(axis=-1)

test_predictions = model.predict(test_generator)
test_true_labels = test_generator.classes
test_predicted_labels = test_predictions.argmax(axis=-1)

train_confusion_matrix = confusion_matrix(train_true_labels, train_predicted_labels)
validation_confusion_matrix = confusion_matrix(validation_true_labels, validation_predicted_labels)
test_confusion_matrix = confusion_matrix(test_true_labels, test_predicted_labels)

print("Train Confusion Matrix:")
print(train_confusion_matrix)
print("\nValidation Confusion Matrix:")
print(validation_confusion_matrix)
print("\nTest Confusion Matrix:")
print(test_confusion_matrix)

train_classification_report = classification_report(train_true_labels, train_predicted_labels)
validation_classification_report = classification_report(validation_true_labels, validation_predicted_labels)
test_classification_report = classification_report(test_true_labels, test_predicted_labels)

print("\nTrain Classification Report:")
print(train_classification_report)
print("\nValidation Classification Report:")
print(validation_classification_report)
print("\nTest Classification Report:")
print(test_classification_report)

model.save("drowsiness_detection_model.h5")

8/8 [==============================] - 1s 133ms/step
Train Confusion Matrix:
[[51 87]
 [71 94]]

Validation Confusion Matrix:
[[ 0 34]
 [ 0 41]]

Test Confusion Matrix:
[[ 97   0]
 [156   0]]

Train Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.37      0.39       138
           1       0.52      0.57      0.54       165

    accuracy                           0.48       303
   macro avg       0.47      0.47      0.47       303
weighted avg       0.47      0.48      0.47       303


Validation Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        34
           1       0.55      1.00      0.71        41

    accuracy                           0.55        75
   macro avg       0.27      0.50      0.35        75
weighted avg       0.30      0.55      0.39        75


Test Classification Report:
              precision    recall  f1-score   support

     

/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mohith/anaconda3/l